test of the colab is working or not..

In [2]:
import pandas as pd

importing and merging the csv files .

In [3]:
import pandas as pd
import os

# Define the path
colab_path = r"c:\Users\goyal\OneDrive\Documents\Desktop\DSA\COLAB"

# Load all 4 aadhar biometric files
file1 = pd.read_csv(f"{colab_path}/api_data_aadhar_biometric_0_500000.csv")
file2 = pd.read_csv(f"{colab_path}/api_data_aadhar_biometric_500000_1000000.csv")
file3 = pd.read_csv(f"{colab_path}/api_data_aadhar_biometric_1000000_1500000.csv")
file4 = pd.read_csv(f"{colab_path}/api_data_aadhar_biometric_1500000_1861108.csv")

# Merge all files
merged_data = pd.concat([file1, file2, file3, file4], ignore_index=True)

print(f"Total rows merged: {len(merged_data)}")
print(f"Columns: {merged_data.columns.tolist()}")
print("\nFirst few rows:")
print(merged_data.head())

Total rows merged: 1861108
Columns: ['date', 'state', 'district', 'pincode', 'bio_age_5_17', 'bio_age_17_']

First few rows:
         date              state      district  pincode  bio_age_5_17  \
0  01-03-2025            Haryana  Mahendragarh   123029           280   
1  01-03-2025              Bihar     Madhepura   852121           144   
2  01-03-2025  Jammu and Kashmir         Punch   185101           643   
3  01-03-2025              Bihar       Bhojpur   802158           256   
4  01-03-2025         Tamil Nadu       Madurai   625514           271   

   bio_age_17_  
0          577  
1          369  
2         1091  
3          980  
4          815  


Loading the master pincode data 

In [4]:
# Load master pincode data
master_pincode = pd.read_csv(f"{colab_path}/india_pincode_master.csv")

# Remove duplicates - keep first occurrence of each pincode
master_pincode_dedup = master_pincode.drop_duplicates(subset=['pincode'], keep='first')

print(f"Master pincode records: {len(master_pincode_dedup)}")
print("\nSample master data:")
print(master_pincode_dedup.head())

Master pincode records: 19586

Sample master data:
   pincode                district      state
0   504273  KUMURAM BHEEM ASIFABAD  TELANGANA
1   504299  KUMURAM BHEEM ASIFABAD  TELANGANA
3   504296  KUMURAM BHEEM ASIFABAD  TELANGANA
6   504209              MANCHERIAL  TELANGANA
9   504272              MANCHERIAL  TELANGANA


Cell 3

In [5]:
# Merge merged_data with master_pincode using pincode as key
cleaned_data = merged_data.merge(
    master_pincode_dedup[['pincode', 'district', 'state']], 
    on='pincode', 
    how='left',
    suffixes=('_old', '_correct')
)

# Replace old state and district with correct ones from master
cleaned_data['state'] = cleaned_data['state_correct'].fillna(cleaned_data['state_old'])
cleaned_data['district'] = cleaned_data['district_correct'].fillna(cleaned_data['district_old'])

# Drop unnecessary columns
cleaned_data = cleaned_data.drop(['state_old', 'district_old', 'state_correct', 'district_correct'], axis=1)

print(f"Cleaned data shape: {cleaned_data.shape}")
print("\nCleaned data:")
print(cleaned_data.head(10))

Cleaned data shape: (1861108, 6)

Cleaned data:
         date  pincode  bio_age_5_17  bio_age_17_              state  \
0  01-03-2025   123029           280          577            HARYANA   
1  01-03-2025   852121           144          369              BIHAR   
2  01-03-2025   185101           643         1091  JAMMU AND KASHMIR   
3  01-03-2025   802158           256          980              BIHAR   
4  01-03-2025   625514           271          815         TAMIL NADU   
5  01-03-2025   416702           155          529        MAHARASHTRA   
6  01-03-2025   388130            75          143            GUJARAT   
7  01-03-2025   382421           192          298            GUJARAT   
8  01-03-2025   759025           122          214             ODISHA   
9  01-03-2025   396055            67           85            GUJARAT   

       district  
0  MAHENDRAGARH  
1     MADHEPURA  
2        POONCH  
3       BHOJPUR  
4       MADURAI  
5     RATNAGIRI  
6         ANAND  
7     AHMADABAD

In [6]:
# Show records where state/district was corrected
print("Sample of corrected records:")
print(cleaned_data.head(20))

# Check for any NULL values
print(f"\nNull values in cleaned data:\n{cleaned_data.isnull().sum()}")

# Save cleaned data
output_path = f"{colab_path}/cleaned_merged_data.csv"
cleaned_data.to_csv(output_path, index=False)
print(f"\nCleaned data saved to: {output_path}")

Sample of corrected records:
          date  pincode  bio_age_5_17  bio_age_17_              state  \
0   01-03-2025   123029           280          577            HARYANA   
1   01-03-2025   852121           144          369              BIHAR   
2   01-03-2025   185101           643         1091  JAMMU AND KASHMIR   
3   01-03-2025   802158           256          980              BIHAR   
4   01-03-2025   625514           271          815         TAMIL NADU   
5   01-03-2025   416702           155          529        MAHARASHTRA   
6   01-03-2025   388130            75          143            GUJARAT   
7   01-03-2025   382421           192          298            GUJARAT   
8   01-03-2025   759025           122          214             ODISHA   
9   01-03-2025   396055            67           85            GUJARAT   
10  01-03-2025   636119           103           63         TAMIL NADU   
11  01-03-2025   712414            91           97        WEST BENGAL   
12  01-03-2025   72114

In [7]:
# Summary statistics
print("="*60)
print("DATA CLEANING SUMMARY REPORT")
print("="*60)
print(f"\nTotal Records Processed: {len(cleaned_data)}")
print(f"Total Columns: {len(cleaned_data.columns)}")
print(f"\nColumns: {cleaned_data.columns.tolist()}")

print(f"\nUnique States: {cleaned_data['state'].nunique()}")
print(f"Unique Districts: {cleaned_data['district'].nunique()}")
print(f"Unique Pincodes: {cleaned_data['pincode'].nunique()}")

print("\nState Distribution:")
print(cleaned_data['state'].value_counts())

DATA CLEANING SUMMARY REPORT

Total Records Processed: 1861108
Total Columns: 6

Columns: ['date', 'pincode', 'bio_age_5_17', 'bio_age_17_', 'state', 'district']

Unique States: 70
Unique Districts: 1120
Unique Pincodes: 19707

State Distribution:
state
TAMIL NADU         182486
MAHARASHTRA        149052
UTTAR PRADESH      144214
KARNATAKA          138534
ANDHRA PRADESH     131078
                    ...  
Orissa                 28
Goa                    27
Manipur                 5
Westbengal              5
Jammu & Kashmir         1
Name: count, Length: 70, dtype: int64
